In [15]:
import folium

class Map:
    def __init__(self, center, zoom):
        self.center = center
        self.zoom = zoom
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def remove_layer(self, layer):
        self.layers.remove(layer)
    
    def set_zoom(self, zoom):
        self.zoom = zoom
    
    def set_center(self, center):
        self.center = center
    
    def create_marcador(self,lat,lon,titulo):
        marker = folium.Marker(location=[lat, lon], popup=titulo)
        return marker
    
    def show(self):
        m = folium.Map(location=[self.center[0], self.center[1]], zoom_start=self.zoom)
        for layer in self.layers:
            m.add_child(layer)
        return m

## VISUALIZAR MAPA

In [21]:
#VISUALIZAR MAPA
m = Map(center=[-22.9035, -43.2096], zoom=10)
lat=-20.9035 
lon=-43.2096
popup='Hello, world!'
marker = m.creat_marcador(lat,lon,popup)
m.add_layer(marker)
lat=-22.9035 
lon=-43.2096
popup='hehe'
marker = m.creat_marcador(lat,lon,popup)
m.add_layer(marker)
m.set_zoom(5)
m.show()

In [27]:
import math

class Coordinate:
    def __init__(self, lat, lon, alt=None):
        self.latitude = lat
        self.longitude = lon
        self.altitude = alt
        
    def distance(self, other, algorithm='euclidean'):
        # calcula a distância entre esta coordenada e outra, usando o algoritmo especificado
        if algorithm == 'euclidean':
            # fórmula da distância euclidiana entre dois pontos
            return math.sqrt((self.latitude - other.latitude)**2 + (self.longitude - other.longitude)**2)
        elif algorithm == 'haversine':
            # fórmula de Haversine para distância entre dois pontos em uma esfera (como a Terra)
            R = 6371  # raio médio da Terra em quilômetros
            d_lat = math.radians(other.latitude - self.latitude)
            d_lon = math.radians(other.longitude - self.longitude)
            lat1 = math.radians(self.latitude)
            lat2 = math.radians(other.latitude)
            a = math.sin(d_lat/2)**2 + math.sin(d_lon/2)**2 * math.cos(lat1) * math.cos(lat2)
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
            return R * c
        else:
            raise ValueError(f"Unknown distance algorithm: {algorithm}")


## CALCULA DISTANCIA ENTRE DOIS PONTOS

In [28]:
# criando duas coordenadas
p1 = Coordinate(lat=-22.906847, lon=-43.172897)
p2 = Coordinate(lat=-23.5505, lon=-46.6333)

# calculando a distância de Haversine entre as duas coordenadas
dist_haversine = p1.distance(p2, algorithm='haversine')
print(f"Distância de Haversine: {dist_haversine:.2f} km")

Distância de Haversine: 360.75 km


In [14]:
import requests

class Location:
    def __init__(self, name, address, latitude, longitude):
        self.name = name
        self.address = address
        self.latitude = latitude
        self.longitude = longitude
        self.temperature = None
        self.humidity = None
        self.pressure = None

    def set_weather(self, api_key,lat,lon):
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
        response = requests.get(url)
        data = response.json()
        
        if 'main' in data:
            self.temperature = data["main"]["temp"]
            self.humidity = data["main"]["humidity"]
            self.pressure = data["main"]["pressure"]
        else:
            self.temperature = None
            self.humidity = None
            self.pressure = None

    def get_coordinates(self):
        return self.latitude, self.longitude


## DADOS METEROLOGICOS DA LOCALIZAÇÃO

In [15]:
# cria um objeto Location para a cidade de Nova York
lat =-23.5505 
lon=-46.6333
nyc = Location("São Paulo", "São Paulo, Brasil", lat,lon)

# obtém as informações meteorológicas para a cidade de Nova York
nyc.set_weather('1fad30734db859e67fe4b73213276fbf',lat ,lon)

# exibe as informações meteorológicas
print(f"Temperatura de {nyc.name}: {nyc.temperature}°C")
print(f"Humidade de {nyc.name}: {nyc.humidity}%")
print(f"Pressão de {nyc.name}: {nyc.pressure} hPa")


Temperatura de São Paulo: 298.6°C
Humidade de São Paulo: 75%
Pressão de São Paulo: 999 hPa
